In [ ]:
def Acc_BSUM_Dict_Learning(Y, dims, lamda, max_iter = 150, A_init = None, X_init = None):
  D, N, K = dims
  A = generateRandomDict(D, K) if A_init is None else np.copy(A_init)
  X = rng.standard_normal(size = (K, N)) if X_init is None else np.copy(X_init)

  objective_values = np.zeros(max_iter, dtype = np.float64)
  recon_error = np.zeros(max_iter, dtype = np.float64)
  # objective_values[0] = d(Y, A, X)
  # recon_error[0] = cal_recon_error(Y, A, X)

  A_prev = np.copy(A)
  w = np.zeros_like(A_prev)
  i = 0
  theta = np.zeros(max_iter, dtype = np.float64)
  clf = LassoLars(alpha = lamda / D, normalize = False)
  while(i < max_iter):
    theta[i] = 2 / (i + 2)
    V_r = (1 - theta[i-1]) * A_prev + theta[i] * w
    clf.fit(V_r, Y)
    X = clf.coef_.T
    # tau_x = linalg.svdvals(V_r)[0] ** 2
    # X = S(X - V_r.T @ (V_r @ X - Y) / tau_x , lamda / tau_x)
    tau_a = linalg.svdvals(X)[0] ** 2
    A_cur = P(V_r - (V_r @ X - Y) @ X.T / (tau_a))
    w = A_prev + (A_cur - A_prev) / theta[i]

    objective_values[i] = d(Y, A_cur, X)
    recon_error[i] = cal_recon_error(Y, A_cur, X)

    i += 1

  return A_cur, X, objective_values, recon_error